In [41]:
# import necessay modules
import torch
import torchinfo
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from torch.utils.data import Dataset, DataLoader # Gives easier dataset managment by creating mini batches etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from tqdm import tqdm # for nice 

In [26]:
# device config
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 

In [43]:
# Hyperparameter
in_channel = 3 # for minist 28*28
num_classes = 10 # 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [44]:
# Load Data
# shape=3, 32, 32
train_dataset = datasets.CIFAR10(
    root="./data", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.CIFAR10(
    root="./data", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [45]:
for batch_idx, (data, targets) in enumerate((train_loader)):
    print(data.shape)
    if(batch_idx==2): break

torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])


In [74]:
# create CNN layer:
class CNNNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes=10):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels= 6, kernel_size=(5,5)) # output = (32-5+2*0)/1+1=28 -> [1, 6,28,28]
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) # 
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels= 16, kernel_size=(5,5))
        self.conv3 = nn.Conv2d(16,120,5)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(120,64)
        self.fc2 = nn.Linear(64,num_classes)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        # x = x.reshape(x.shape[0], -1)
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = CNNNet().to(device)
torchinfo.summary(model, input_size=(1, 3, 32, 32), col_names= ("input_size", "output_size", "num_params", "mult_adds"), verbose=2,)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
CNNNet                                   [1, 3, 32, 32]            [1, 10]                   --                        --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 6, 28, 28]            456                       357,504
│    └─weight                                                                                ├─450
│    └─bias                                                                                  └─6
├─MaxPool2d: 1-2                         [1, 6, 28, 28]            [1, 6, 14, 14]            --                        --
├─Conv2d: 1-3                            [1, 6, 14, 14]            [1, 16, 10, 10]           2,416                     241,600
│    └─weight                                                                                ├─2,400
│    └─bias                                                                 

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
CNNNet                                   [1, 3, 32, 32]            [1, 10]                   --                        --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 6, 28, 28]            456                       357,504
│    └─weight                                                                                ├─450
│    └─bias                                                                                  └─6
├─MaxPool2d: 1-2                         [1, 6, 28, 28]            [1, 6, 14, 14]            --                        --
├─Conv2d: 1-3                            [1, 6, 14, 14]            [1, 16, 10, 10]           2,416                     241,600
│    └─weight                                                                                ├─2,400
│    └─bias                                                                 

In [68]:
# Initialize network
model = CNNNet().to(device)
torchinfo.summary(model, input_size=(1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
CNNNet                                   [1, 10]                   --
├─Conv2d: 1-1                            [1, 6, 30, 30]            456
├─MaxPool2d: 1-2                         [1, 6, 15, 15]            --
├─Conv2d: 1-3                            [1, 16, 11, 11]           2,416
├─MaxPool2d: 1-4                         [1, 16, 5, 5]             --
├─Conv2d: 1-5                            [1, 120, 1, 1]            48,120
├─Linear: 1-6                            [1, 64]                   7,744
├─Linear: 1-7                            [1, 10]                   650
Total params: 59,386
Trainable params: 59,386
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.76
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31

In [9]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train Network
def train(num_epochs, train_loader, model, device, loss_fn, optimizer):
    for epoch in range(num_epochs):
        losses=[] 
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # Forward
            scores = model(data)
            loss = loss_fn(scores, targets)
            losses.append(loss.item())

            # Backward
            optimizer.zero_grad()
            loss.backward()

            # Gradient descent or adam step
            optimizer.step()
        mean_loss=sum(losses)/len(losses)
        print(f"at {epoch} epoch loss: {mean_loss:.5f}")

In [13]:
train(num_epochs, train_loader, model, device, loss_fn, optimizer)

100%|██████████| 938/938 [00:27<00:00, 34.73it/s]


at 0 epoch loss: 0.32642


100%|██████████| 938/938 [00:21<00:00, 43.03it/s]


at 1 epoch loss: 0.09283


100%|██████████| 938/938 [00:25<00:00, 37.17it/s]

at 2 epoch loss: 0.06818


In [11]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [12]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 9.93
Accuracy on test set: 10.31
